References:

https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

https://github.com/pytorch/text/blob/master/examples/legacy_tutorial/migration_tutorial.ipynb

https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f

In [42]:
import torch
from torch.utils.data import dataset
from torch import nn, Tensor
import spacy

nlp = spacy.load("en_core_web_sm")

use_gpu = torch.cuda.is_available()
if use_gpu:
  print("CUDA available.\nPytorch with GPU processing")
  DEVICE = torch.device("cuda")
  result = spacy.require_gpu()
  print("Spacy set with GPU " if result else None)
else:
  print("CUDA not available. CPU processing")
  DEVICE = torch.device("cpu")
DEVICE

CUDA available.
Pytorch with GPU processing
Spacy set with GPU 


device(type='cuda')

In [43]:
import pandas as pd
from torch.utils.data import Dataset

In [44]:
class ReviewsDataset(Dataset):
    def __init__(self, reviews_file):
        self.df = pd.read_csv(reviews_file)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        review = self.df.iloc[idx, 0]
        starts = self.df.iloc[idx, 1]
        return review, starts

    def review(self, idx):
        return self.df.iloc[idx, 0]

    def stars(self, idx):
        return self.df.iloc[idx, 1]

In [45]:
label_type = 'final_label'

In [46]:
train_iter = ReviewsDataset(f'../dataset/{label_type}/train.csv')
val_iter = ReviewsDataset(f'../dataset/{label_type}/train.csv')
test_iter = ReviewsDataset(f'../dataset/{label_type}/test.csv')

In [47]:
SEED = 1234

import random
import numpy as np

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [48]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for review, stars in data_iter:
        yield tokenizer(review)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [49]:
vocab(['here', 'is', 'an', 'example'])

[286, 12, 39, 266]

In [50]:
rows_iter = iter(train_iter)


In [51]:
next(rows_iter)

('Four Stars. thought provoking', 1)

In [52]:
tokens_iter = iter(yield_tokens(train_iter))

In [53]:
next(tokens_iter)

['four', 'stars', '.', 'thought', 'provoking']

### Vocabulary size:

In [54]:
len(vocab)

8159

In [55]:
# vocab_spacy = {}

# for index,row in enumerate(iter(train_iter)):
#     doc = nlp(row[0])
#     sentences = doc.sents

#     for sent in sentences:
#         tokens = nlp(sent.text)

#     for tkn in tokens:
#         if tkn.text in vocab_spacy.keys():
#             vocab_spacy[tkn.text] += 1
#         else:
#             vocab_spacy[tkn.text] = 1
# len(vocab_spacy)

vocab_spacy: 4653 (takes 5 min to process with GPU enabled)

In [56]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: x

In [57]:
text_pipeline('here is the an example')

[286, 12, 3, 39, 266]

In [58]:
text_pipeline('This one is a weird one for me to write')

[15, 48, 12, 8, 1586, 48, 18, 58, 6, 791]

In [59]:
text_pipeline("<pad>")

[1]

In [60]:
PAD_IDX = vocab(['<pad>'])[0]
PAD_IDX

1

In [61]:
# Number of batches for training
train_batch_size = 64

# Number of batches for validation. Use a larger value than training.
# It helps speed up the validation process.
valid_batch_size = 100

In [62]:
# def batchify(tensors: Tuple, batch_size: int) -> Tensor:
#     """Divides the data into bacthes separate sequences, removing extra elements
#     that wouldn't cleanly fit.

#     Args:
#         data: Tensor, shape [N]
#         bsz: int, batch size

#     Returns:
#         Tensor of shape [N // batch_size, batch_size]
#     """
#     texts = tensors[0]
#     labels = tensors[1]
#     seq_len = texts.__len__() // batch_size
#     texts = texts[:seq_len * batch_size]
#     texts = texts.view(batch_size, seq_len).t().contiguous()

#     seq_len = labels.__len__() // batch_size
#     labels = labels[:seq_len * batch_size]
#     labels = labels.view(batch_size, seq_len).t().contiguous()
#     return texts.to(DEVICE), labels.to(DEVICE)

In [63]:
# def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
#     """Converts raw text into a flat Tensor."""
    
#     # [print(type(item[1])) for item in raw_text_iter]
#     texts = [torch.tensor(text_pipeline(item[0]), dtype=torch.long) for item in raw_text_iter]
#     labels = [torch.tensor(item[1], dtype=torch.long) for item in raw_text_iter]
#     return torch.cat(tuple(filter(lambda t: t.numel() > 0, texts))), labelslabels)))

In [64]:
# train_iterator = batchify(data_process(training_data), train_batch_size) 
# val_iterator = batchify(data_process(valid_data), valid_batch_size)
# test_iterator = batchify(data_process(test_data), valid_batch_size)

In [65]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
   label_list, text_list = [], []
   for (_text, _label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text))
         text_list.append(processed_text)
   return pad_sequence(text_list, padding_value=PAD_IDX), torch.tensor(label_list)

In [66]:
train_list = list(train_iter)
batch_size = train_batch_size 

def batch_sampler():
    indices = [(i, len(tokenizer(s[0]))) for i, s in enumerate(train_list)]
    random.shuffle(indices)
    pooled_indices = []
    # create pool of indices with similar lengths 
    for i in range(0, len(indices), batch_size * 100):
        pooled_indices.extend(sorted(indices[i:i + batch_size * 100], key=lambda x: x[0]))

    pooled_indices = [x[0] for x in pooled_indices]

    # yield indices for current batch
    for i in range(0, len(pooled_indices), batch_size):
        yield pooled_indices[i:i + batch_size]

bucket_dataloader = DataLoader(train_list, batch_sampler=batch_sampler(), collate_fn=collate_batch)

print(next(iter(bucket_dataloader)))

(tensor([[ 204,   24,   46,  ...,  351, 3299,  325],
        [  70,   45,   96,  ...,   28,  334,  638],
        [   2,    9,   21,  ...,    2,    7,   10],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]]), tensor([1, 0, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 0, 2, 2,
        1, 0, 2, 2, 1, 2, 2, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 1, 2,
        2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1]))


In [67]:
train_dataloader = DataLoader(list(train_iter), batch_sampler=batch_sampler(), collate_fn=collate_batch)
val_dataloader = DataLoader(list(val_iter), batch_sampler=batch_sampler(), collate_fn=collate_batch)
test_dataloader = DataLoader(list(test_iter), batch_sampler=batch_sampler(), collate_fn=collate_batch)

In [68]:
print(next(iter(train_dataloader)))

(tensor([[ 204,   24,   46,  ...,  351, 3299,  325],
        [  70,   45,   96,  ...,   28,  334,  638],
        [   2,    9,   21,  ...,    2,    7,   10],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]]), tensor([1, 0, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 0, 2, 2,
        1, 0, 2, 2, 1, 2, 2, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 1, 2,
        2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1]))


In [69]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 100, 
                                out_channels = 16, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 16, 
                                out_channels = 32, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 32, 
                                out_channels = 16, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        # conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        # conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        # pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        # pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0)#, pooled_1, pooled_2)
                                    , dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [70]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

FastText Embeddings

In [71]:
# from torchtext.vocab import FastText
# embedding = FastText('simple')

CharNGram Embeddings

In [72]:
# from torchtext.vocab import CharNGram
# embedding_charngram = CharNGram()

GloVe Embeddings

In [73]:
# from torchtext.vocab import GloVe
# embedding_glove = GloVe(name='6B', dim=100)

In [74]:
# EMBEDDING_LAYER = torch.nn.Embedding.from_pretrained(embedding.vectors,freeze=False)

In [75]:
# EMBEDDING_LAYER.embedding_dim

In [76]:
# myvocab = vocab(EMBEDDING_LAYER.stoi)

In [77]:
# EMBEDDING_LAYER.stoi['<unk>']

In [78]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 3
DROPOUT = 0.5

In [79]:
# embedding = embedding
# embedding = embedding_charngram
# embedding = embedding_glove

In [80]:
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
# model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [81]:
if use_gpu:
    print("Trying to use GPU")
    import torch.backends.cudnn as cudnn
    torch.cuda.init()
    cudnn.benchmark = True
    model.cuda()
model

Trying to use GPU


CNN(
  (embedding): Embedding(8159, 100, padding_idx=1)
  (conv_0): Conv2d(100, 16, kernel_size=(3, 100), stride=(1, 1))
  (conv_1): Conv2d(16, 32, kernel_size=(4, 100), stride=(1, 1))
  (conv_2): Conv2d(32, 16, kernel_size=(5, 100), stride=(1, 1))
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [83]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,757,667 trainable parameters


In [84]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [85]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [86]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [87]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch[0]).squeeze(1)
        
        loss = criterion(predictions, batch[1])
        
        acc = categorical_accuracy(predictions, batch[1])
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [88]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            print(batch)
            predictions = model(batch[0]).squeeze(1)
            
            loss = criterion(predictions, batch[1])
            
            acc = categorical_accuracy(predictions, batch[1])

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [89]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [104]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_dataloader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

TypeError: forward() missing 1 required positional argument: 'actual_batch_len'

## LSTM

In [91]:
class SentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim,
                 hidden, n_label, n_layers):
        super(SentimentClassifier, self).__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden,
                            num_layers=n_layers,
                            bidirectional=True,
                            batch_first=True)#dropout=0.2
        self.fc = nn.Linear(hidden * 2, n_label)
    def forward(self, input, actual_batch_len):
        embed_out = self.embed(input)
        hidden = torch.zeros(self.n_layers * 2 ,
                             input.shape[0], self.hidden)
        cell = torch.zeros( self.n_layers * 2,
                            input.shape[0], self.hidden)
        pack_out = nn.utils.rnn.pack_padded_sequence(
            embed_out, actual_batch_len,batch_first=True).to(device)
        out_lstm, (hidden, cell) = self.lstm(pack_out,
                                             (hidden, cell))#dropout
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]),dim=1)
        out = self.fc(hidden)
        return out

In [92]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 100
HIDDEN= 64
NUM_LABEL = 4 # number of classes
NUM_LAYERS = 2 
model = SentimentClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN, NUM_LABEL, NUM_LAYERS)

In [98]:
def accuracy(preds, y):
    _, preds = torch.max(preds, dim= 1)
    acc = torch.sum(preds == y) / len(y)
    return acc
def calculateLoss(model, batch, criterion):
        text, text_len = batch
        preds = model(text, text_len.to('cpu') )
        loss = criterion(preds, batch.label)
        acc = accuracy(preds, batch.label)
        return loss, len(batch.label), acc

In [99]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model.to(DEVICE)

SentimentClassifier(
  (embed): Embedding(8159, 100)
  (lstm): LSTM(100, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)

In [100]:
N_EPOCH = 100
for i in range (N_EPOCH):
    model.train()
    train_len, train_acc, train_loss  = 0, [], []
    for batch_no, batch in enumerate(train_dataloader):
        opt.zero_grad()
        loss, blen, acc = calculateLoss( model, batch,
                          criterion)
        train_loss.append(loss * blen)
        train_acc.append(acc * blen)
        train_len = train_len + blen
        loss.backward()
        opt.step()
    train_epoch_loss = np.sum(train_loss) / train_len
    train_epoch_acc = np.sum( train_acc ) / train_len
    model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            val_results = [calculateLoss( model, batch,
                                          criterion)
                           for batch in val_dataloader]
            loss, batch_len, acc = zip(*val_results)
            epoch_loss = np.sum(np.multiply(loss, batch_len)) / np.sum(batch_len)
            epoch_acc = np.sum(np.multiply(acc , batch_len)) / np.sum(batch_len)
        print('epoch:{}/{} epoch_train_loss:{:.4f},epoch_train_acc:{:.4f}'
              ' epoch_val_loss:{:.4f},epoch_val_acc:{:.4f}'.format(i+1, N_EPOCH,
                train_epoch_loss.item(), train_epoch_acc.item(),
                epoch_loss.item(), epoch_acc.item()))

RuntimeError: Expected `len(lengths)` to be equal to batch_size, but got 64 (batch_size=130)

In [93]:
for batch in train_dataloader:
    print(batch[0])
    print(batch.count)
    text, len = batch[0], len(batch)
    emb = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
    # emb.weight.data.copy_(TEXT.vocab.vectors)
    emb_out = emb(text)
    pack_out = nn.utils.rnn.pack_padded_sequence(emb_out,
                                                 len,
                                                 batch_first=True)
    rnn = nn.RNN(EMBEDDING_DIM, 4, batch_first=True)
    out, hidden = rnn(pack_out)

tensor([[ 71,  54,   9,  ..., 769, 306,  21],
        [ 23,  25, 120,  ...,   2,  70,  16],
        [  8,  28,  11,  ...,  29,   2,  30],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])
<built-in method count of tuple object at 0x000002140C3BE580>


RuntimeError: 'lengths' argument should be a 1D CPU int64 tensor, but got 0D cuda:0 Long tensor

## BERT

In [101]:
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url

padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256
xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

text_transform = T.Sequential(
    T.SentencePieceTokenizer(xlmr_spm_model_path),
    T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path)),
    T.Truncate(max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
)

100%|██████████| 5.07M/5.07M [00:01<00:00, 4.77MB/s]
Downloading: "https://download.pytorch.org/models/text/xlmr.vocab.pt" to C:\Users\dav/.cache\torch\hub\checkpoints\xlmr.vocab.pt
100%|██████████| 4.85M/4.85M [00:01<00:00, 4.31MB/s]


In [103]:
from torchtext.datasets import SST2
from torch.utils.data import DataLoader
batch_size = 16

train_datapipe = train_iter
dev_datapipe = val_iter

# Transform the raw dataset using non-batched API (i.e apply transformation line by line)
train_datapipe = train_datapipe.map(lambda x: (text_pipeline(x[0]), x[1]))
train_datapipe = train_datapipe.batch(batch_size)
train_datapipe = train_datapipe.rows2columnar(["token_ids", "target"])
train_dataloader = DataLoader(train_datapipe, batch_size=None)

dev_datapipe = dev_datapipe.map(lambda x: (text_pipeline(x[0]), x[1]))
dev_datapipe = dev_datapipe.batch(batch_size)
dev_datapipe = dev_datapipe.rows2columnar(["token_ids", "target"])
dev_dataloader = DataLoader(dev_datapipe, batch_size=None)

AttributeError: 'ReviewsDataset' object has no attribute 'map'